### Task 1. Implementing class Money.

In [1]:
class Money:
    def __init__(self, rubles: int = 0, kopecks: int = 0):
        self._rubles = rubles
        self._kopecks = kopecks
        self.normalize()

    @property
    def rubles(self):
        return self._rubles

    @rubles.setter
    def rubles(self, value: int):
        if not isinstance(value, int):
            raise TypeError('Rubles must be a integer.')
        self._rubles = value
        
    @property
    def kopecks(self):
        return self._kopecks
    
    @kopecks.setter
    def kopecks(self, value: int):
        if not isinstance(value, int):
            raise TypeError('Kopecks must be a integer.')
        self._kopecks = value

    def normalize(self) -> None:
        """
        Current function saves the logic for normalizing the amount of rubles and kopecks.
        :return: None. just provides normalization logic for rubles and kopecks.
        """
        if self.kopecks >= 100:
            self.rubles += self.kopecks // 100
            self.kopecks = self.kopecks % 100
        elif self.kopecks < 0:
            self.rubles -= (-self.kopecks) // 100 + 1
            self.kopecks = 100 - (-self.kopecks) % 100

    def __add__(self, other) -> object:
        return Money(self.rubles + other.rubles, self.kopecks + other.kopecks)

    def __sub__(self, other) -> object:
        return Money(self.rubles - other.rubles, self.kopecks - other.kopecks)

    def __mul__(self, factor) -> object:
        total_kopecks = (self.rubles * 100 + self.kopecks) * factor
        rubles = int(total_kopecks // 100)
        kopecks = int(total_kopecks % 100)
        return Money(rubles, kopecks)

    def __truediv__(self, divisor) -> object:
        total_kopecks = self.rubles * 100 + self.kopecks
        result_kopecks = total_kopecks / divisor
        rubles = int(result_kopecks // 100)
        kopecks = int(result_kopecks % 100)
        return Money(rubles, kopecks)

    def __floordiv__(self, other) -> object:
        total_kopecks_self = self.rubles * 100 + self.kopecks
        total_kopecks_other = other.rubles * 100 + other.kopecks
        return total_kopecks_self // total_kopecks_other

    def __eq__(self, other) -> bool:
        return self.rubles == other.rubles and self.kopecks == other.kopecks

    def __ne__(self, other) -> bool:
        return not self == other

    def __lt__(self, other) -> bool:
        return (self.rubles, self.kopecks) < (other.rubles, other.kopecks)

    def __le__(self, other) -> bool:
        return (self.rubles, self.kopecks) <= (other.rubles, other.kopecks)

    def __gt__(self, other) -> bool:
        return (self.rubles, self.kopecks) > (other.rubles, other.kopecks)

    def __ge__(self, other) -> bool:
        return (self.rubles, self.kopecks) >= (other.rubles, other.kopecks)

    def __str__(self) -> str:
        return f'{self.rubles},{self.kopecks:02d}'

In [2]:
print(first_money := Money(5, 30))

5,30


In [3]:
print(second_money := Money(10, 80))

10,80


In [4]:
print(first_money + second_money)

16,10


In [5]:
first_money > second_money

False

### Task 2. Implementing class Matrix.

In [6]:
from typing import List


class Matrix:
    def __init__(self, data: List[List[int]]):
        
        if not all(len(row) == len(data[0]) for row in data):
            raise ValueError("All rows must have the same number of columns")
        
        self._data = data
        
    @property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value: List[List[int]]):
        if not isinstance(value, list):
            raise TypeError('Data must be a list.')
        self._data = value

    def size(self):
        """        
        :return: Returns the size of the matrix.
        """
        return len(self.data), len(self.data[0])

    def __add__(self, other):
        
        if self.size() != other.size():            
            raise ValueError("Matrices must have the same dimensions to be added")
        
        result = [
            [self.data[i][j] + other.data[i][j] for j in range(len(self.data[0]))]
            for i in range(len(self.data))
        ]
        
        return Matrix(result)

    def __sub__(self, other):
        
        if self.size() != other.size():
            raise ValueError("Matrices must have the same dimensions to be subtracted")
        
        result = [
            [self.data[i][j] - other.data[i][j] for j in range(len(self.data[0]))]
            for i in range(len(self.data))
        ]
        
        return Matrix(result)

    def __mul__(self, other):
        
        if isinstance(other, (int, float)):
            result = [
                [self.data[i][j] * other for j in range(len(self.data[0]))]
                
                for i in range(len(self.data))
            ]
            
            return Matrix(result)
        
        elif isinstance(other, Matrix):
            
            if self.size()[1] != other.size()[0]:
                raise ValueError("Matrices must have compatible dimensions to be multiplied")
            
            result = [
                [
                    sum(self.data[i][k] * other.data[k][j] for k in range(self.size()[1]))
                    for j in range(other.size()[1])
                ]
                for i in range(self.size()[0])
            ]
            
            return Matrix(result)
        
        else:
            raise ValueError("Unsupported operand type(s) for *: 'Matrix' and '{}'".format(type(other)))

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.data])

    def determinant(self):
        """
        :return: Returns the determinant of the matrix.        
        """
        if self.size()[0] != self.size()[1]:
            raise ValueError("Determinant can only be calculated for square matrices")
        
        return self._determinant_recursive(self.data)

    def _determinant_recursive(self, matrix):
        """
        Recursive function for determinant calculation.
        :param matrix: matrix to calculate the determinant of.
        :return: determinant of the matrix.
        """
        if len(matrix) == 1:
            return matrix[0][0]
        
        if len(matrix) == 2:
            return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
        
        det = 0
        
        for c in range(len(matrix)):
            det += ((-1) ** c) * matrix[0][c] * self._determinant_recursive(self._minor(matrix, 0, c))
            
        return det

    def _minor(self, matrix, i, j):
        """
        Current function is used to calculate the minor of the matrix.
        :param matrix: matrix to calculate the minor of.
        :param i: index of the row to calculate the minor of.
        :param j: index of the column to calculate the minor of.
        :return: minor of the matrix.
        """
        return [row[:j] + row[j + 1:] for row in (matrix[:i] + matrix[i + 1:])]

    def inverse(self):
        """        
        :return: the inverse of the matrix.
        """
        det = self.determinant()
        
        if det == 0:
            return None
        
        size = self.size()[0]
        
        if size == 1:
            return Matrix([[1 / self.data[0][0]]])
        
        cofactors = [
            [((-1) ** (i + j)) * self._determinant_recursive(self._minor(self.data, i, j)) for j in range(size)]
            for i in range(size)
        ]
        
        cofactors = Matrix(cofactors).transpose().data
        
        return Matrix([[cofactors[i][j] / det for j in range(size)] for i in range(size)])

    def transpose(self):
        """
        Calculates the transpose of the matrix.
        :return: Transpose of the matrix.
        """
        transposed = [[self.data[j][i] for j in range(len(self.data))] for i in range(len(self.data[0]))]
        
        return Matrix(transposed)

In [7]:
print(matrix_one := Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]]))

1 2 3
4 5 6
7 8 9


In [8]:
print(matrix_two := Matrix([[10, 11, 12], [13, 14, 15], [16, 17, 18]]))

10 11 12
13 14 15
16 17 18


In [9]:
print(matrix_one.transpose())

1 4 7
2 5 8
3 6 9


In [10]:
print(matrix_one.inverse())

None


In [11]:
print(matrix_one + matrix_two)

11 13 15
17 19 21
23 25 27
